In [266]:

#read intput.txt
lines = []
with open('input1.txt') as f:
    # split text by empty line
    for line in f.read().split('\n\n'):
        lines.append(line.replace('\n', '|').split('|'))

seeds = [int(s) for s in lines[0][0].split(':')[1].strip().split(' ')]

def create_map(lines):
    maps = []
    for entry in lines[1:]:
        key = entry[0].replace(' map:', '')
        values = [v.split(" ") for v in entry[1:]]
        values = [{"from":int(v[1]), "to":int(v[0]), "range":int(v[2])} for v in values]
        maps.append({key: values})
    return maps


In [267]:

def map_seed_to_location(maps,seed):
    value = seed
    for m in maps:
        rule_sets = m.values()
        for rules in rule_sets:
            for r in rules:
                if r['from'] <= value <= r['from']+r['range']:
                    value = r['to'] + (value - r['from'])
                    break
    return value

def solve_part1(maps, seeds):
    locations = []
    for seed in seeds:
        locations.append(map_seed_to_location(maps, seed))
    print("Part1:", min(locations))

maps = create_map(lines)
solve_part1(maps, seeds)

Part1: 35


In [276]:
# groub seeds in tupples of even and odd e.g (1,2),(3,4),(5,6)
import portion as P

seed_rages = [(seeds[i], seeds[i] + seeds[i+1] - 1) for i in range(0, len(seeds), 2)]
intervals = P.empty()
for sr in seed_rages:
    intervals = intervals | P.closed(sr[0], sr[1])

print(intervals)

for m in maps:
    rule_sets = m.values()
    for rules in rule_sets:
        new_intervals = set([])
        print("##################")
        print("intervals:", intervals)
        for ru in rules:
            rule_interval = P.closed(ru['from'], ru['from']+ru['range'] - 1)
            print(rule_interval, ru['to'] - ru['from'])
        for interval in intervals:
            for r in rules:
                rule_interval = P.closed(r['from'], r['from']+r['range'] - 1)
                shift = r['to'] - r['from']
                intersection = None
                rest = None
                if interval.overlaps(rule_interval):
                    intersection = interval & rule_interval
                    rest = interval - intersection
                    new_interval = intersection.apply(lambda x: (x.left, x.lower + shift, x.upper + shift, x.right))
                    new_intervals.add(new_interval)
                    if (interval in new_intervals):
                        new_intervals.remove(interval)
                    if rest: 
                        intervals = intervals | rest
                        print("rest:", rest)
                    print("new_intervals:", new_intervals, "inteval", interval)
                    break
                else:
                    new_intervals.add(interval)
                # print("interval:", interval, "rule_interval:", rule_interval, "intersection:", intersection, "rest:", rest, "shift:", shift)
                print("new_intervals:", new_intervals, "inteval", interval)
        intervals = P.empty()
        for interval in new_intervals:
            intervals = intervals | interval

min_interval = min(intervals, key=lambda i: i.lower)
print(intervals)
print("Part2:", min_interval.lower)

[55,67] | [79,92]
##################
intervals: [55,67] | [79,92]
[98,99] -48
[50,97] 2
new_intervals: {[55,67]} inteval [55,67]
new_intervals: {[57,69]} inteval [55,67]
new_intervals: {[57,69], [79,92]} inteval [79,92]
new_intervals: {[57,69], [81,94]} inteval [79,92]
##################
intervals: [57,69] | [81,94]
[15,51] -15
[52,53] -15
[0,14] 39
new_intervals: {[57,69]} inteval [57,69]
new_intervals: {[57,69]} inteval [57,69]
new_intervals: {[57,69]} inteval [57,69]
new_intervals: {[57,69], [81,94]} inteval [81,94]
new_intervals: {[57,69], [81,94]} inteval [81,94]
new_intervals: {[57,69], [81,94]} inteval [81,94]
##################
intervals: [57,69] | [81,94]
[53,60] -4
[11,52] -11
[0,6] 42
[7,10] 50
rest: (60,69]
new_intervals: {[53,56]} inteval [57,69]
new_intervals: {[53,56], [81,94]} inteval [81,94]
new_intervals: {[53,56], [81,94]} inteval [81,94]
new_intervals: {[53,56], [81,94]} inteval [81,94]
new_intervals: {[53,56], [81,94]} inteval [81,94]
##################
intervals: 